In [123]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel

from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    SystemMessagePromptTemplate,
)
from langchain.prompts import load_prompt, ChatPromptTemplate
from langchain.schema import AIMessage, HumanMessage, BaseMessage

from langchain_core.callbacks import BaseCallbackHandler

In [124]:
load_dotenv()

True

In [125]:
API_URL = "http://localhost:8000"

In [126]:
def unpack_messages(messages):
    unpacked = ""
    for message in messages:
        if isinstance(message, HumanMessage):
            unpacked += f"User: {message.content}\n"
        elif isinstance(message, AIMessage):
            unpacked += f"AI: {message.content}\n"
        # Add more conditions here if you're using other message types
    return unpacked


In [127]:
# llm = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature=1.2, model_kwargs={"top_p": 0.5})
# llm = ChatOpenAI(model_name = "gpt-4", temperature=1.2, model_kwargs={"top_p": 0.5})
llm = ChatOpenAI(model_name = "gpt-4", temperature=1.2)

SYSTEM_THOUGHT = load_prompt('prompts/thought.yaml')
SYSTEM_RESPONSE = load_prompt('prompts/response.yaml')
SYSTEM_USER_PREDICTION_THOUGHT = load_prompt('prompts/user_prediction_thought.yaml')

system_thought = SystemMessagePromptTemplate(prompt=SYSTEM_THOUGHT)
system_response = SystemMessagePromptTemplate(prompt=SYSTEM_RESPONSE)
system_user_prediction_thought = SystemMessagePromptTemplate(prompt=SYSTEM_USER_PREDICTION_THOUGHT)

In [134]:
messages = {
	"thoughts": [],
	"responses": [],
	"user_prediction_thought": []
}

Wonderings:
- Why does the `think` step only consider the previous `thoughts` and not `responses`?

In [129]:
def think(query: str):
	"""Generate Bloom's thought on the user."""

	thought_prompt = ChatPromptTemplate.from_messages([
		system_thought,
		*messages["thoughts"],
		HumanMessage(content=query)
	])

	chain = thought_prompt | llm 


	class SaveMessagesHandler(BaseCallbackHandler):
		def on_llm_end(self, response, **kwargs):
			ai_response = response.generations[0][0].text

			messages["thoughts"].append(HumanMessage(content=query))
			messages["thoughts"].append(AIMessage(content=ai_response))

	save_messages_handler = SaveMessagesHandler()

	return chain.invoke({}, config={
		"callbacks": [save_messages_handler]
	})

In [130]:
def respond(thought: str, query: str):
	"""Generate Bloom's response to the user."""

	response_prompt = ChatPromptTemplate.from_messages([
		system_response,
		*messages["responses"],
		HumanMessage(content=query)
	])

	chain = response_prompt | llm 


	class SaveMessagesHandler(BaseCallbackHandler):
		def on_llm_end(self, response, **kwargs):
			ai_response = response.generations[0][0].text

			messages["responses"].append(HumanMessage(content=query))
			messages["responses"].append(AIMessage(content=ai_response))

	save_messages_handler = SaveMessagesHandler()

	return chain.invoke({"thought": thought}, config={
		"callbacks": [save_messages_handler]
	})

Wonderings:
- Where is `user_prediction_thought` used??

In [131]:
def think_user_prediction():
	"""Generate a thought about what the user is going to say"""

	prompt = ChatPromptTemplate.from_messages([
		system_user_prediction_thought,
	])

	chain = prompt | llm

	history = unpack_messages(messages["responses"])

	user_prediction_thought = chain.invoke({"history": history})

	messages["user_prediction_thought"].append(user_prediction_thought)


In [132]:
def chat(query: str ) -> tuple[str, str]:
	thought = think(query=query)

	response = respond(thought=thought, query=query)

	think_user_prediction()

	return thought, response


In [135]:
chat("I'm not able to understand the difference between constructionism and constructivism")

(AIMessage(content='User appears confused with distinguishing philosophical and educational concepts, like constructivism and constructionism. Predicted need: References or resources with clear explanations to explain the distinction and possibly similar foundational theories that surround these concepts.\n\nAdditional helpful data for future predictions: user’s academic background, previous exposure to or experience with pedagogic concepts, and understanding of educational principles. This will provide contextual backgrounds to tailor the explanations.', response_metadata={'token_usage': {'completion_tokens': 80, 'prompt_tokens': 84, 'total_tokens': 164}, 'model_name': 'gpt-4', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}),
 AIMessage(content="Constructionism and constructivism are indeed quite similar because they're both educational theories about how people learn, but they focus on slightly different aspects. \n\nConstructivism is about how individuals cre